In [ ]:
# # Setup
# ! sudo apt install -y libgl1-mesa-glx libglib2.0-0 libsm6 libxrender1 libxext6
# ! pip install open-iris==1.0.0 faiss-cpu seaborn

# Imports, Constants, and Base Functions

In [ ]:
import boto3
from io import BytesIO
import pickle
import iris
import scipy
import psutil
import time
from datetime import datetime
import sys
import threading
from itertools import combinations, product
from functools import reduce
from operator import mul
from joblib import Parallel, delayed
from scipy.spatial.distance import pdist

In [ ]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp, ttest_ind

In [ ]:
n_jobs = 6 # Fit to CPU
DIM = (2, 32, 200)
X, Y = DIM [1:]
MAX_ROT = 15

In [ ]:
single_sample_size = 5000
gaussian_base_path = 'data/voter_param_search_gaussian_low_init/'
uniform_base_path = 'data/voter_param_search_uniform_init/'

In [ ]:
def plot_boolean_iris(matrix, title=''):
    plt.imshow(matrix, cmap='gray')
    plt.title(title)
    plt.show()

In [ ]:
def import_data(path, base_path=uniform_base_path, num_samples=single_sample_size, X=X//2, Y=Y):
    return (
        np.unpackbits(np.fromfile(base_path+path, dtype=np.uint8), bitorder="little")
        .reshape(num_samples, X, Y)
    )

# Real Irises

## Raw Data Loading

In [ ]:
shape = (16, 200)
# DEV = "-dev" # Access test data.
DEV = "" # Access real data.
print("Working on simulated data" if DEV else "Working on real data")

In [ ]:
bucket_name = 'wld-inversed-data-sharing' + DEV
role_arn = 'arn:aws:iam::387760840988:role/worldcoin-data' + DEV
metadata_path = 'metadata.csv'

def memoize(func):
    cache = {}
    def memoized_func(*args):
        if args in cache:
            return cache[args]
        result = func(*args)
        cache[args] = result
        return result
    return memoized_func

def assume_role(role_arn, session_name="S3ReadSession"):
    sts_client = boto3.client('sts')
    assumed_role_object = sts_client.assume_role(
        RoleArn=role_arn,
        RoleSessionName=session_name
    )
    credentials = assumed_role_object['Credentials']
    s3 = boto3.client(
        's3',
        aws_access_key_id=credentials['AccessKeyId'],
        aws_secret_access_key=credentials['SecretAccessKey'],
        aws_session_token=credentials['SessionToken']
    )
    return s3

# Assume the role and get credentials
s3 = assume_role(role_arn, "S3ReadSession")

def read_s3_file(bucket_name, file_key):
    obj = s3.get_object(Bucket=bucket_name, Key=file_key)
    return BytesIO(obj['Body'].read())

@memoize
def load_response(image_id):
    " Return IrisFilterResponse "
    path = "iris_filter_responses/" + image_id + ".pickle"
    try:
        pkl_file = read_s3_file(bucket_name, path)
        return pickle.load(pkl_file)
    except Exception as err:
        print(err)
        return None

@memoize
def load_template(image_id):
    " Return IrisTemplate "
    path = "iris_templates/" + image_id + ".pickle"
    try:
        pkl_file = read_s3_file(bucket_name, path)
        return pickle.load(pkl_file)
    except Exception as err:
        print(err)
        return None

# Read the file into a DataFrame
meta = pd.read_csv(read_s3_file(bucket_name, metadata_path))

## Data Processing

### Functions

In [ ]:
# Helpers for iterators.
def take(count, it):
    " Take at most `count` items from the iterator `it` "
    for x in it:
        if count is not None:
            if count <= 0:
                break
            count -= 1
        yield x

In [ ]:
# Load matching pairs.
def iter_matching_image_ids(meta, unique_subjects):
    " Iterate matching pairs in the form (subject_id, ir_image_id_0, ir_image_id_1). "
    subject_ids = meta["subject_id"].unique()

    for side in [0, 1]:
        meta_side = meta[meta["biological_side"] == side]

        for subject in subject_ids:
            signups = meta_side[meta_side["subject_id"] == subject]
            if len(signups) < 2:
                continue

            L = 2 if unique_subjects else len(signups)

            for i in range(L - 1):
                for j in range(i + 1, L):
                    yield (f"{subject}_side{side}", signups["ir_image_id"].iloc[i], signups["ir_image_id"].iloc[j])

def load_matching_image_ids(meta, unique_subjects):
    " Return matching pairs in the form (subject_id, ir_image_id_0, ir_image_id_1), shuffled. "    
    pair_image_ids = list(iter_matching_image_ids(meta, unique_subjects))
    rng = np.random.default_rng(seed=12345)
    rng.shuffle(pair_image_ids)
    return pair_image_ids

def iter_related_pairs(meta, unique_subjects):
    " Iterate matching pairs in the form (subject_id, response_0, response_1). "
    for (subject, img_i, img_j) in load_matching_image_ids(meta, unique_subjects):
        res_i = load_response(img_i)
        res_j = load_response(img_j)
        if res_i and res_j:
            yield (subject, res_i, res_j)

def load_related_pairs(meta, count=None, unique_subjects=False):
    " Return matching pairs in the form (subject_id, response_0, response_1). "
    return list(take(count, iter_related_pairs(meta, unique_subjects)))

In [ ]:
# Masking methodologies
def fill_masked_with_random(bits, mask):
    filler = np.random.randint(0, 2, size=bits.shape, dtype=bool)
    filler &= not_(mask)
    bits ^= filler

def fill_masked_with_zeros(bits, mask):
    bits &= mask

# Techniques that do not support masking will work, although with a modified scale of distances.
# The change in distance can be calculated from the size of the overlap of masks. Alternatively,
# it can be estimated with the expected average of that.

In [ ]:
# Make encoders from parameters.
def make_encoder(v_subsample=1, h_subsample=1, top=True, bottom=True, real=True, imag=True, mask_threshold=0.9, static_mask=None, mask_with_random=False):

    res_indexes = (top and [0] or []) + (bottom and [1] or [])
    assert res_indexes, "require top, bottom, or both"

    quantizers = (real and [np.real] or []) + (imag and [np.imag] or [])
    assert quantizers, "require real, imag, or both"
    
    def encode(response):
        bit_parts = []
        mask_parts = []
        
        for res_index in res_indexes:
            for quantizer in quantizers:
                res = response.iris_responses[res_index][::v_subsample, ::h_subsample]
                bits = quantizer(res) > 0
                mask = response.mask_responses[res_index][::v_subsample, ::h_subsample] >= mask_threshold

                if mask_with_random:
                    # Replace masked bits with random bits.
                    fill_masked_with_random(bits, mask)
                
                if static_mask is not None:
                    # Remove the bits not selected by the static mask.
                    fill_masked_with_zeros(bits, static_mask[::v_subsample, ::h_subsample])
                    # Treat non-selected bits as masked (False).
                    mask &= static_mask[::v_subsample, ::h_subsample]
                
                bit_parts.append(bits)
                mask_parts.append(mask)
                assert mask.shape == bits.shape

        return np.concatenate(bit_parts), np.concatenate(mask_parts)
    
    return encode

def encode_pairs(pairs, encode_fn):
    return [
        (subject_id, encode_fn(response_a), encode_fn(response_b))
        for subject_id, response_a, response_b in pairs
    ]

In [ ]:
# Distances
def masked_distance(x, x_mask, y, y_mask):
    mask = x_mask & y_mask
    hd = np.sum((x ^ y) & mask)
    return hd / np.sum(mask)

def masked_rotate(x, rotation):
    return (
        np.roll(x[0], rotation, axis=1),
        np.roll(x[1], rotation, axis=1),
    )

def distance(x, y):
    return masked_distance(x[0], x[1], y[0], y[1])

def distance_raw(raw_x, raw_y):
    return distance(encode_high(raw_x), encode_high(raw_y))

def rotate_raw(raw_x, rotation):
    iris_responses = [
        np.roll(r, rotation, axis=1)
        for r in raw_x.iris_responses
    ]
    mask_responses = [
        np.roll(r, rotation, axis=1)
        for r in raw_x.mask_responses
    ]
    return iris.IrisFilterResponse(iris_responses=iris_responses, mask_responses=mask_responses)

In [ ]:
# Rotations.
def without_rotation(pairs, distance_fn, rotate_fn, max_rotation):
    for subject_id, x, y in pairs:
        distances = [
            distance_fn(x, rotate_fn(y, rotation))
            for rotation in range(-max_rotation, max_rotation+1)
        ]
        best_rotation = -max_rotation + np.argmin(distances)        
        y_aligned = rotate_fn(y, best_rotation)
        yield (subject_id, x, y_aligned)

def remove_rotation(pairs, distance_fn=distance, rotate_fn=masked_rotate, max_rotation=15):
    return list(without_rotation(pairs, distance_fn, rotate_fn, max_rotation))

In [ ]:
def plot_boolean_iris(matrix, title=''):
    plt.imshow(matrix, cmap='gray')
    plt.title(title)
    plt.show()

### Loading

In [ ]:
# 6 min
encode_high = make_encoder()
related_pairs = load_related_pairs(meta, count=None, unique_subjects=False)
related_pairs_norot = remove_rotation(related_pairs, distance_fn=distance_raw, rotate_fn=rotate_raw)
related_pairs_high = encode_pairs(related_pairs_norot, encode_high)
shape_high = related_pairs_high[0][1][0].shape
print(f"Finished loading {len(related_pairs_high)} pairs,", "High-res", shape_high, np.prod(shape_high), "bits")

In [ ]:
tuples_array = np.array(related_pairs_high, dtype=object)
subject_ids = np.repeat(tuples_array[:, 0], 2)  # Repeat each subject_id twice
flattened_result = [item for tup in tuples_array for item in tup[1:]]
iris_matrices, mask_matrices = zip(*flattened_result)

In [ ]:
true_iris_df = pd.DataFrame({
    'subject_id': subject_ids,
    'iris_matrices': iris_matrices,
    'mask_matrices': mask_matrices
})
true_iris_df['side'] = true_iris_df['subject_id'].apply(lambda x: x[-1])
true_iris_df['subject_id'] = true_iris_df['subject_id'].apply(lambda x: x.split('_')[0])

In [ ]:
# Dropping duplicates
true_iris_df['iris_matrices_bytes'] = true_iris_df['iris_matrices'].apply(lambda matrix: matrix.tobytes())
true_iris_df['mask_matrices_bytes'] = true_iris_df['mask_matrices'].apply(lambda matrix: matrix.tobytes())
true_iris_df = (
    true_iris_df
    .drop_duplicates(subset=['subject_id', 'iris_matrices_bytes', 'mask_matrices_bytes'])
    .drop(columns=['iris_matrices_bytes', 'mask_matrices_bytes'])
    .reset_index(drop=True)
)
print(f'Final iris DataFrame contains {len(true_iris_df)} different samples')

In [ ]:
unique_true_iris_df = true_iris_df.drop_duplicates(subset=['subject_id', 'side'])
print(f'Unique iris DataFrame contains {len(unique_true_iris_df)} samples')

# Synthetic Data Quality Tests

## Rotation Test

In [ ]:
alpha = 0.05
MAX_ROT = 30

In [ ]:
real_iris_rotations_distance_dict = dict()
for rot in range(-MAX_ROT, MAX_ROT+1):
    if rot == 0:
        continue
    low_rotations, high_rotations = zip(
        *true_iris_df['iris_matrices'].apply(
            lambda matrix: [np.sum(part != np.roll(part, shift=rot, axis=1)) / part.size for part in np.split(matrix, 2, axis=0)]
        )
    )
    real_iris_rotations_distance_dict[rot] = high_rotations

In [ ]:
results = []
for base_path in [uniform_base_path]: # [gaussian_base_path, uniform_base_path]
    base_legend = pd.read_csv(base_path+'manifest.csv')
    for index, (filename, num_samples, iterations, bias, init_method) in base_legend.iterrows():
        data = import_data(path=filename, base_path=base_path, num_samples=num_samples)
        for rot in range(-MAX_ROT, MAX_ROT+1):
            if rot == 0:
                continue
            rolled_data = np.roll(data, shift=rot, axis=2)
            rotated_distances = (np.sum(data != rolled_data, axis=(1, 2)) / ((X//2) * Y))
            
            ks_stat, ks_p_value = ks_2samp(rotated_distances, real_iris_rotations_distance_dict[rot]) # Kolmogorov-Smirnov test
            t_stat, t_p_value = ttest_ind(rotated_distances, real_iris_rotations_distance_dict[rot]) # Student's t test
                
            results.append({
                'Rotation':rot,
                'Source':init_method.replace('_',' ')+f', {iterations} iterations, {bias} bias',
                'ks_stat':ks_stat,
                't_stat':t_stat,
                'ks_p_value':ks_p_value,
                't_p_value':t_p_value,
                'mean':rotated_distances.mean(),
                'std':rotated_distances.std(),
            })

results_df = pd.DataFrame(results)
results_df['passed_KS_test'] = results_df['ks_p_value'] >= alpha
results_df['passed_t_test'] = results_df['t_p_value'] >= alpha

In [ ]:
statistical_tests_results_df = (
    results_df
    .groupby('Source')[['ks_p_value', 'passed_KS_test', 't_p_value', 'passed_t_test']]
    .mean()
    .reset_index()
    .sort_values(['ks_p_value', 't_p_value'], ascending=False)
)
statistical_tests_results_df.head()

In [ ]:
any_reach_peak = results_df.groupby('Source')['mean'].max() > 0.51
any_reach_peak[any_reach_peak].index

In [ ]:
statistical_tests_results_df = (
    results_df[results_df['Rotation'] <= 6] # Before the "peak"
    .groupby('Source')[['ks_p_value', 'passed_KS_test', 't_p_value', 'passed_t_test']]
    .mean()
    .reset_index()
    .sort_values(['ks_p_value', 't_p_value'], ascending=False)
)
statistical_tests_results_df.head()

In [ ]:
real_irises_rotations_df = pd.DataFrame(
    [
        pd.Series(
            {
                'mean':np.mean(real_iris_rotations_distance_dict[rot]),
                'std':np.std(real_iris_rotations_distance_dict[rot]),
                'Source':'real irises',
                'Rotation':rot
            }
        ) for rot in range(-MAX_ROT, MAX_ROT+1) if rot != 0
    ]
).melt(
    id_vars=['Rotation', 'Source'],
    value_vars=['mean', 'std'],
    var_name='Metric',
)

In [ ]:
potential_sources = statistical_tests_results_df.head()['Source'].tolist()
plot_df = pd.melt(
    results_df[results_df['Source'].isin(potential_sources)],
    id_vars=['Rotation', 'Source'],
    value_vars=['mean', 'std'],
    var_name='Metric',
)
plot_df = pd.concat([plot_df, real_irises_rotations_df])

In [ ]:
for source in potential_sources:
    mask = plot_df['Source'].isin(['real irises']+[source])
    facetgrid = sns.FacetGrid(plot_df[mask], col='Metric', height=5, aspect=2, sharex=False, sharey=False)
    facetgrid.map_dataframe(sns.lineplot, x='Rotation', y='value', hue='Source', palette='husl')
    [(ax.grid(True), ax.legend()) for ax in facetgrid.axes.flat]
    facetgrid.fig.suptitle(f"Mean and Std of real and {source.replace('_', ' ')} iris samples, in relation to rotation", fontsize=15, y=1.05)
    plt.show()

In [ ]:
MAX_ROT = 15

## Boolean Ratio Test

In [ ]:
ratio_df_lst = []
for base_path in [uniform_base_path]: # [gaussian_base_path, uniform_base_path]
    base_legend = pd.read_csv(base_path+'manifest.csv')
    for index, (filename, num_samples, iterations, bias, init_method) in base_legend.iterrows():
        data = import_data(path=filename, base_path=base_path, num_samples=num_samples)
        ratio_df_lst.append(
            pd.DataFrame(
                pd.Series(data.mean(axis=(1,2))).rename('Boolean Ratio')
            ).assign(
                Source = init_method.replace('_',' '),
                Iterations = iterations,
                Bias = bias,
            )
        )
ratio_df = pd.concat(ratio_df_lst).melt(id_vars=['Boolean Ratio', 'Source'], value_vars=['Iterations', 'Bias'], var_name='Feature')
ratio_df.head()

In [ ]:
facetgrid = sns.FacetGrid(ratio_df, col='Feature', hue='Source', palette='husl', sharex=False, sharey=False, height=4, aspect=1.8)
facetgrid.map_dataframe(sns.lineplot, x='value', y='Boolean Ratio')
facetgrid.fig.suptitle(f"True / False Ratio Validation", fontsize=20, y=1.2)
facetgrid.add_legend()
plt.show()

## Nearest to Random Dist Test

In [ ]:
def stack_rotated_matrices(matrices, max_rotation):
    return np.vstack([
        np.roll(matrix, shift, axis=0).flatten()
        for matrix in matrices
        for shift in range(-max_rotation, max_rotation + 1)
    ])

def get_min_and_random_dist_across_rotations(iris_matrices, mask_matrices, max_rotation):
    num_matrices = len(iris_matrices)
    num_rotations = 2 * max_rotation + 1

    # Rotate matrices and masks, reshape to (num_matrices, num_rotations, flattened_size)
    rotated_matrices = stack_rotated_matrices(iris_matrices, max_rotation).reshape(num_matrices, num_rotations, -1)
    rotated_masks = stack_rotated_matrices(mask_matrices, max_rotation).reshape(num_matrices, num_rotations, -1)

    closest_distances, random_distances = [], []
    for i in range(num_matrices):
        # Current matrix rotations and masks
        current_rotated_matrix = rotated_matrices[i]
        current_rotated_mask = rotated_masks[i]

        # Extract other matrices' rotations excluding the current
        other_rotated_matrices = np.delete(rotated_matrices, i, axis=0).reshape(-1, rotated_matrices.shape[-1])
        other_rotated_masks = np.delete(rotated_masks, i, axis=0).reshape(-1, rotated_masks.shape[-1])

        # Calculate valid positions and Hamming distances
        valid_positions = current_rotated_mask[:, None] & other_rotated_masks
        differences = current_rotated_matrix[:, None] != other_rotated_matrices

        # Calculate Hamming distances
        hamming_distances = np.sum(differences & valid_positions, axis=-1) / np.sum(valid_positions, axis=-1)
        
        # Find the minimum distance and a random distance
        closest_distances.append(np.min(hamming_distances))
        random_distances.append(np.random.choice(hamming_distances.flatten()))

    return pd.DataFrame({"closest_dist": closest_distances, "random_dist": random_distances})

def load_and_reshape_masks(filename, num_masks):
    flattened_data = np.unpackbits(np.fromfile(filename, dtype=np.uint8))
    boolean_arrays = flattened_data.reshape((num_masks, X//2, Y))
    vertically_stacked = np.tile(boolean_arrays, (1, 2, 1))
    # duplicated_arrays = np.repeat(vertically_stacked[:, np.newaxis, :, :], DIM[0], axis=1) # For full irises 
    return vertically_stacked

def int_to_scaled_string(n):
    suffixes = ['', 'K', 'M', 'B', 'T']
    idx = max(0, min(len(suffixes) - 1, int((len(str(abs(n))) - 1) / 3)))
    scaled = n / (1000 ** idx)
    return f"{scaled:.1f}{suffixes[idx]}" if scaled % 1 else f"{int(scaled)}{suffixes[idx]}"

def process_file(filename, num_samples, iterations, bias, init_method, base_path, num_unique_samples, loaded_masks):
    data = import_data(path=filename, base_path=base_path, num_samples=num_samples)
    random_indices = np.random.choice(num_samples, size=2*num_unique_samples, replace=False)
    sampled_data = np.take(data, random_indices, axis=0).reshape(num_unique_samples, X, Y)
    random_indices = np.random.choice(num_samples, size=num_unique_samples, replace=False)
    sampled_masks = np.take(loaded_masks, random_indices, axis=0)
    return (
        get_min_and_random_dist_across_rotations(sampled_data, sampled_masks, max_rotation=MAX_ROT)
        .assign(Source=init_method.replace('_', ' ') + f', {iterations} iterations, {bias} bias')
    )

In [ ]:
low_wavelets, high_wavelets = np.split(np.stack(unique_true_iris_df['iris_matrices'].values), 2, axis=1)
low_wavelets_masks, high_wavelets_masks = np.split(np.stack(unique_true_iris_df['mask_matrices'].values), 2, axis=1)

In [ ]:
real_irises_dist = get_min_and_random_dist_across_rotations(high_wavelets, high_wavelets_masks, max_rotation=MAX_ROT).assign(Source = 'Real Irises')

In [ ]:
num_masks = 1000000
path = f'data/synthetic_iris_data/{int_to_scaled_string(num_masks)}_mask_arrays.dat'
loaded_masks = load_and_reshape_masks(path, num_masks)

In [ ]:
results = []
for base_path in [uniform_base_path]: # [gaussian_base_path, uniform_base_path]
    base_legend = pd.read_csv(base_path + 'manifest.csv')
    parallel_results = Parallel(n_jobs=n_jobs)(delayed(process_file)(
        filename, single_sample_size, iterations, bias, init_method, base_path, len(unique_true_iris_df), loaded_masks
    ) for index, (filename, num_samples, iterations, bias, init_method) in base_legend.iterrows())
    results.extend(parallel_results)
results_df = pd.concat([real_irises_dist]+[x for x in results if x is not None])

In [ ]:
best_results = results_df.groupby('Source')['closest_dist'].mean().sort_values().index[:6]
plot_df = pd.melt(
    results_df[results_df['Source'].isin(best_results)], 
    id_vars='Source', 
    value_vars=['random_dist', 'closest_dist'],
    var_name='Distance From',
    value_name='Hamming Distance'
)
plot_df['Distance From'] = plot_df['Distance From'].apply(lambda x: x.split('_')[0].capitalize())

In [ ]:
facetgrid = sns.FacetGrid(plot_df, col='Distance From', hue='Source', palette='husl', height=4, aspect=2, sharex=False, sharey=False)
facetgrid.map_dataframe(sns.histplot, x='Hamming Distance', stat='probability', kde=True)
[ax.grid(True) for ax in facetgrid.axes.flat]
facetgrid.add_legend()
facetgrid.fig.suptitle(f"Distance from Random / Nearest iris (only low wavelets)\nVarious Data Sources", fontsize=20, y=1.2)
plt.show()

## Comparing pair-wise distance distributions (to Daugman survey)

In [ ]:
def pairwise_distance_wrapper(filename, num_samples, iterations, bias, init_method, base_path, num_real_irises=len(unique_true_iris_df)):
    data = import_data(path=filename, base_path=base_path, num_samples=num_samples)
    random_indices = np.random.choice(num_samples, size=num_real_irises, replace=False)
    distances = pdist(np.take(data, random_indices, axis=0).reshape(num_real_irises, -1), metric='hamming')
    return pd.DataFrame({
        'Distances':distances,
        'Source':init_method.replace('_', ' ') + f', {iterations} iterations, {bias} bias',
    })

In [ ]:
low_wavelets, high_wavelets = np.split(np.stack(unique_true_iris_df['iris_matrices'].values), 2, axis=1)
real_iris_matrices = high_wavelets.reshape(len(unique_true_iris_df), -1)
real_pairwise_distances = pdist(real_iris_matrices, metric='hamming')
real_iris_df = pd.DataFrame({
    'Distances':real_pairwise_distances,
    'Source':'Real Irises',
})

In [ ]:
pairwise_results = []
for base_path in [uniform_base_path]: # [gaussian_base_path, uniform_base_path]
    base_legend = pd.read_csv(base_path + 'manifest.csv')
    parallel_results = Parallel(n_jobs=n_jobs)(delayed(pairwise_distance_wrapper)(
        filename, single_sample_size, iterations, bias, init_method, base_path
    ) for index, (filename, num_samples, iterations, bias, init_method) in base_legend.iterrows())
    pairwise_results.extend(parallel_results)
pairwise_results_df = pd.concat([real_iris_df]+[x for x in pairwise_results if x is not None])
pairwise_results_df.head()

In [ ]:
potential_sources = abs(pairwise_results_df.groupby('Source')['Distances'].mean() - pairwise_results_df.loc[pairwise_results_df['Source'] == 'Real Irises', 'Distances'].mean()).sort_values()
potential_sources = potential_sources[:6].index

In [ ]:
plt.figure(figsize=(10,8))
sns.histplot(pairwise_results_df[pairwise_results_df['Source'].isin(potential_sources)], x='Distances', stat='probability', hue='Source', palette='husl', kde=True)
plt.grid(True)
plt.title('Pairwise Distance Distribution', fontsize=15, y=1.07)
plt.show()

# Visually view several samples

In [ ]:
base_legend = pd.read_csv(uniform_base_path + 'manifest.csv')
filename = '5k_voter_arrays_14k_b010_un.dat'
data = import_data(path=filename)
random_indices = np.random.choice(single_sample_size, size=4, replace=False)
for i in random_indices:
    plot_boolean_iris(np.vstack((data[i], data[i+1])))